# Teste técnico Dhauz

Vamos iniciar autenticando as credenciais

In [1]:
!pip install mysql-connector-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import mysql.connector
import sys
import boto3
import os

ENDPOINT="dhauz-instance.cutloqirhpd7.us-east-1.rds.amazonaws.com"
PORT="3306"
USER="candidate_user"
REGION="us-east-1"
DBNAME="db_hiring_test"
pswrd = "D3@bGh664%$1VHv*"
os.environ['LIBMYSQL_ENABLE_CLEARTEXT_PLUGIN'] = '1'

In [4]:
#gets the credentials from .aws/credentials
session = boto3.Session()
client = session.client('rds', region_name=REGION)


/usr/local/lib/python3.7/dist-packages/botocore/client.py:631: FutureWarning: The rds client is currently using a deprecated endpoint: rds.amazonaws.com. In the next minor version this will be moved to rds.us-east-1.amazonaws.com. See https://github.com/boto/botocore/issues/2705 for more details.
  category=FutureWarning,


# Análise inicial do banco de dados

Para começar, iremos realizar uma consulta incluindo todos os resultados

In [144]:
try:
    conn =  mysql.connector.connect(host=ENDPOINT, user=USER, passwd=pswrd, port=PORT, database=DBNAME, ssl_ca='SSLCERTIFICATE')
    cur = conn.cursor()
    cur.execute("""SELECT *, REPLACE(TotalSent,',','') as Value FROM db_hiring_test.raw_transactions_table LIMIT 10""")
    query_results = cur.fetchall()
    field_names = print([i[0] for i in cur.description])
    for row in query_results:
      print(row)
except Exception as e:
    print("Database connection failed due to {}".format(e))
                

['IdTransaction', 'AddressOrigin', 'AddressDestination', 'TotalSent', 'Status', 'SentDate', 'ImportDate', 'Value']
('ID1002', 'A-77', 'A-49', ' 293,659.00 ', 'Confirmed', '2021-01-08 13:34:04', '2021-01-31 23:59:59', ' 293659.00 ')
('ID2014', 'A-24', 'A-58', ' 542,285.00 ', 'Confirmed', '2021-01-17 13:34:04', '2021-01-31 23:59:59', ' 542285.00 ')
('ID1092', 'A-15', 'A-20', ' 57,493.00 ', 'Confirmed', '2021-01-03 03:07:57', '2021-01-31 23:59:59', ' 57493.00 ')
('ID1603', 'A-84', 'A-59', ' 883,745.00 ', 'Confirmed', '2021-01-02 06:36:39', '2021-01-31 23:59:59', ' 883745.00 ')
('ID253', 'A-86', 'A-44', ' 194,591.00 ', 'Confirmed', '2021-01-14 20:22:08', '2021-01-31 23:59:59', ' 194591.00 ')
('ID322', 'A-62', 'A-18', ' 130,057.00 ', 'Confirmed', '2021-01-01 06:14:33', '2021-01-31 23:59:59', ' 130057.00 ')
('ID2330', 'A-24', 'A-71', ' 469,855.00 ', 'Confirmed', '2021-01-09 21:45:26', '2021-01-31 23:59:59', ' 469855.00 ')
('ID2732', 'A-3', 'A-50', ' 998,501.00 ', 'Denied', '2021-01-20 16:29:

# Fase 1:

## Questão 1: 
 Começando com as consultas que respondem as questões levantadas

 Primeiramente pegamos as 5 primeiras carteiras que enviaram mais transações

In [15]:
try:
    cur2 = conn.cursor()
    cur2.execute("""SELECT AddressOrigin, COUNT(AddressOrigin) FROM db_hiring_test.raw_transactions_table GROUP BY AddressOrigin ORDER BY COUNT(AddressOrigin) DESC LIMIT 5""")
    query_results2 = cur.fetchall()
    for row in query_results2:
      print(row)
except Exception as e:
    print("Database connection failed due to {}".format(e))

Database connection failed due to Unread result found


## Questão 2: Dias com maior incidência de transações



In [12]:
try:
    cur3 = conn.cursor()
    cur3.execute("""SELECT DAYOFMONTH(SentDate) as Dia, COUNT(DAYOFMONTH(SentDate)) FROM db_hiring_test.raw_transactions_table GROUP BY Dia ORDER BY COUNT(Dia) DESC LIMIT 5""")
    query_results3 = cur.fetchall()
    for row in query_results3:
      print(row)
except Exception as e:
    print("Database connection failed due to {}".format(e))

(7, 288)
(2, 284)
(15, 274)
(27, 262)
(12, 262)


## Questão 3: Dia da semana com maior incidência de transferências

In [ ]:
try:
    cur4 = conn.cursor()
    cur4.execute("""SELECT DAYNAME(SentDate) as dia_da_semana, COUNT(DAYNAME(SentDate)) FROM db_hiring_test.raw_transactions_table GROUP BY dia_da_semana ORDER BY COUNT(dia_da_semana) DESC LIMIT 7""")
    query_results4 = cur.fetchall()
    for row in query_results4:
      print(row)
except Exception as e:
    print("Database connection failed due to {}".format(e))

('Saturday', 978)
('Wednesday', 932)
('Friday', 930)
('Thursday', 911)
('Sunday', 874)
('Tuesday', 850)
('Monday', 701)


## Questão 4: Transações suspeitas dentro dos dados

A priori iremos olhar para as transações que tiveram suas condições negadas, e desta forma analisar o que elas podem ter em comum

In [148]:
try:
    cur5 = conn.cursor()
    cur5.execute("""SELECT * FROM db_hiring_test.raw_transactions_table WHERE Status = 'Denied' ORDER BY TotalSent DESC LIMIT 10""")
    query_results5 = cur.fetchall()
    for row in query_results5:
      print(row)
except Exception as e:
    print("Database connection failed due to {}".format(e))

('ID2732', 'A-3', 'A-50', ' 998,501.00 ', 'Denied', '2021-01-20 16:29:47', '2021-01-31 23:59:59')
('ID2732', 'A-3', 'A-50', ' 998,501.00 ', 'Denied', '2021-01-20 16:29:47', '2021-02-05 23:59:59')
('ID3051', 'A-15', 'A-87', ' 988,621.00 ', 'Denied', '2021-02-01 05:41:31', '2021-02-05 23:59:59')
('ID1354', 'A-66', 'A-12', ' 987,415.00 ', 'Denied', '2021-01-05 19:19:47', '2021-02-05 23:59:59')
('ID1354', 'A-66', 'A-12', ' 987,415.00 ', 'Denied', '2021-01-05 19:19:47', '2021-01-31 23:59:59')
('ID3126', 'A-46', 'A-67', ' 831,296.00 ', 'Denied', '2021-02-04 19:16:00', '2021-02-05 23:59:59')
('ID2593', 'A-88', 'A-71', ' 804,143.00 ', 'Denied', '2021-01-09 11:48:41', '2021-02-05 23:59:59')
('ID2593', 'A-88', 'A-71', ' 804,143.00 ', 'Denied', '2021-01-09 11:48:41', '2021-01-31 23:59:59')
('ID3008', 'A-59', 'A-47', ' 694,983.00 ', 'Denied', '2021-02-04 01:35:23', '2021-02-05 23:59:59')
('ID1796', 'A-87', 'A-72', ' 69,019.00 ', 'Denied', '2021-01-02 13:34:04', '2021-02-05 23:59:59')


Podemos reparar a incidência de transações duplicadas e também com datas coincidindo na casa dos segundos, sendo esse o motivo para elas terem seu status negado, sendo passível de investigação

## Questão 5:

Agora para a análise iremos realizar uma função que realiza o cálculo das carteiras

In [188]:
try:
    lista = []
    cur6 = conn.cursor()
    cur6.execute("""WITH pagamentos as (SELECT AddressOrigin as Address, SUM(REPLACE(TotalSent,',','')) as Value FROM db_hiring_test.raw_transactions_table GROUP BY Address),
                    recebimentos as (SELECT AddressDestination as Address, SUM(REPLACE(TotalSent,',','')*-1) as Value FROM db_hiring_test.raw_transactions_table GROUP BY Address)
                    INSERT INTO pagamentos (Address, Value) VALUE(SELECT * FROM recebimentos);
                    SELECT Address, SUM(Value) FROM pagamentos GROUP BY Address""")
    query_results6 = cur.fetchall()
    for row in query_results6:
      print(row)
except Exception as e:
    print("Database connection failed due to {}".format(e))

Database connection failed due to 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'INSERT INTO pagamentos (Address, Value) VALUE(SELECT * FROM recebimentos);
     ' at line 3


# Fase 2:

Agora iremos reunir as informações das duas tabelas para podermos associar o custo de cada transação com a sua respectiva taxa

In [44]:
try:
    cur7 = conn.cursor()
    cur7.execute("""SELECT * FROM db_hiring_test.raw_transactions_fee""")
    query_results7 = cur.fetchall()
    field_names = print([i[0] for i in cur7.description])
    for row in query_results7:
      print(row)
except Exception as e:
    print("Database connection failed due to {}".format(e))

['range-start', 'range-end', 'fee-percentage']
(Decimal('0.00'), Decimal('160000.00'), Decimal('10.00'))
(Decimal('160000.01'), Decimal('340000.00'), Decimal('8.00'))
(Decimal('340000.01'), Decimal('500000.00'), Decimal('6.00'))
(Decimal('500000.01'), Decimal('670000.00'), Decimal('5.00'))
(Decimal('670000.01'), Decimal('833000.00'), Decimal('4.00'))
(Decimal('833000.01'), Decimal('99000000.00'), Decimal('2.00'))


Criando a tabela auxiliar para podermos realizar as análises

In [138]:
 try: 
    cur8 = conn.cursor()
    cur8.execute("""WITH tabAux AS (SELECT * FROM db_hiring_test.raw_transactions_table t1 INNER JOIN db_hiring_test.raw_transactions_fee t2 ON REPLACE(t1.TotalSent,',','') <= t2.`range-end` AND REPLACE(t1.TotalSent,',','') >= t2.`range-start`) 
                    SELECT * FROM tabAux LIMIT 5""") 
    query_results8 = cur.fetchall()
    field_names = print([i[0] for i in cur8.description])
    for row in query_results8:
      print(row)
except Exception as e:
    print("Database connection failed due to {}".format(e))

['IdTransaction', 'AddressOrigin', 'AddressDestination', 'TotalSent', 'Status', 'SentDate', 'ImportDate', 'range-start', 'range-end', 'fee-percentage']
('ID1002', 'A-77', 'A-49', ' 293,659.00 ', 'Confirmed', '2021-01-08 13:34:04', '2021-01-31 23:59:59', Decimal('160000.01'), Decimal('340000.00'), Decimal('8.00'))
('ID2014', 'A-24', 'A-58', ' 542,285.00 ', 'Confirmed', '2021-01-17 13:34:04', '2021-01-31 23:59:59', Decimal('500000.01'), Decimal('670000.00'), Decimal('5.00'))
('ID1092', 'A-15', 'A-20', ' 57,493.00 ', 'Confirmed', '2021-01-03 03:07:57', '2021-01-31 23:59:59', Decimal('0.00'), Decimal('160000.00'), Decimal('10.00'))
('ID1603', 'A-84', 'A-59', ' 883,745.00 ', 'Confirmed', '2021-01-02 06:36:39', '2021-01-31 23:59:59', Decimal('833000.01'), Decimal('99000000.00'), Decimal('2.00'))
('ID253', 'A-86', 'A-44', ' 194,591.00 ', 'Confirmed', '2021-01-14 20:22:08', '2021-01-31 23:59:59', Decimal('160000.01'), Decimal('340000.00'), Decimal('8.00'))


Para realizar a verificação da fórmula, iremos exibir os valores das taxas das 5 primeiras transações

In [120]:
try:    
    cur9 = conn.cursor()
    cur9.execute("""WITH tabAux AS (SELECT * FROM db_hiring_test.raw_transactions_table t1 INNER JOIN db_hiring_test.raw_transactions_fee t2 ON REPLACE(t1.TotalSent,',','') <= t2.`range-end` AND REPLACE(t1.TotalSent,',','') >= t2.`range-start`) 
                    SELECT IdTransaction, AddressOrigin, REPLACE(TotalSent,',','') *`fee-percentage`/100 AS TaxPaid FROM tabAux LIMIT 5""")
    query_results9 = cur.fetchall()
    field_names = print([i[0] for i in cur9.description])
    for row in query_results9:
      print(row)
except Exception as e:
    print("Database connection failed due to {}".format(e))

['IdTransaction', 'AddressOrigin', 'TaxPaid']
('ID1002', 'A-77', 23492.72)
('ID2014', 'A-24', 27114.25)
('ID1092', 'A-15', 5749.3)
('ID1603', 'A-84', 17674.9)
('ID253', 'A-86', 15567.28)


Agora realizaremos a agregação dos valores para notarmos quais são os 5 primeiras carteiras que mais pagaram taxa no mês de Janeiro de 2021

In [128]:
try:
    cur10 = conn.cursor()
    cur10.execute("""WITH tabAux AS (SELECT * FROM db_hiring_test.raw_transactions_table t1 INNER JOIN db_hiring_test.raw_transactions_fee t2 ON REPLACE(t1.TotalSent,',','') <= t2.`range-end` AND REPLACE(t1.TotalSent,',','') >= t2.`range-start`) 
                    SELECT AddressOrigin, SUM(REPLACE(TotalSent,',','') *`fee-percentage`)/100 AS TaxPaid FROM tabAux WHERE MONTH(SentDate) = 01 AND YEAR(SentDate) = 2021 GROUP BY AddressOrigin ORDER BY TaxPaid DESC LIMIT 5""") 
    query_results10 = cur.fetchall()
    field_names = print([i[0] for i in cur10.description])
    for row in query_results10:
      print(row)
except Exception as e:
    print("Database connection failed due to {}".format(e))

['AddressOrigin', 'TaxPaid']
('A-99', 1993046.36)
('A-83', 1765339.54)
('A-25', 1713452.8)
('A-72', 1675731.68)
('A-68', 1668694.14)


Agora realizaremos o mesmo procedimento para Fevereiro de 2021

In [127]:
try:
    cur11 = conn.cursor()
    cur11.execute("""WITH tabAux AS (SELECT * FROM db_hiring_test.raw_transactions_table t1 INNER JOIN db_hiring_test.raw_transactions_fee t2 ON REPLACE(t1.TotalSent,',','') <= t2.`range-end` AND REPLACE(t1.TotalSent,',','') >= t2.`range-start`) 
                    SELECT AddressOrigin, SUM(REPLACE(TotalSent,',','') *`fee-percentage`)/100 AS TaxPaid FROM tabAux WHERE MONTH(SentDate) = 02 AND YEAR(SentDate) = 2021 GROUP BY AddressOrigin ORDER BY TaxPaid DESC LIMIT 5""") 
    query_results11 = cur.fetchall()
    field_names = print([i[0] for i in cur11.description])
    for row in query_results11:
      print(row)
except Exception as e:
    print("Database connection failed due to {}".format(e))

['AddressOrigin', 'TaxPaid']
('A-29', 107830.31)
('A-30', 106344.95)
('A-68', 104406.01)
('A-27', 102267.66)
('A-57', 92955.41)


Podemos também realizar uma querie para pegarmos o valor da maior taxa e o ID da transação associado a ela

In [139]:
try:
    cur12 = conn.cursor()
    cur12.execute("""WITH tabAux AS (SELECT * FROM db_hiring_test.raw_transactions_table t1 INNER JOIN db_hiring_test.raw_transactions_fee t2 ON REPLACE(t1.TotalSent,',','') <= t2.`range-end` AND REPLACE(t1.TotalSent,',','') >= t2.`range-start`) 
                    SELECT *, REPLACE(TotalSent,',','')*`fee-percentage`/100 AS TaxPaid FROM tabAux ORDER BY TaxPaid DESC LIMIT 1""") 
    query_results12 = cur.fetchall()
    field_names = print([i[0] for i in cur12.description])
    for row in query_results12:
      print(row)
except Exception as e:
    print("Database connection failed due to {}".format(e))

['IdTransaction', 'AddressOrigin', 'AddressDestination', 'TotalSent', 'Status', 'SentDate', 'ImportDate', 'range-start', 'range-end', 'fee-percentage', 'TaxPaid']
('ID635', 'A-49', 'A-44', ' 669,701.00 ', 'Confirmed', '2021-01-08 17:59:13', '2021-01-31 23:59:59', Decimal('500000.01'), Decimal('670000.00'), Decimal('5.00'), 33485.05)


Na última questão da Fase 2 iremos pegar a média de todas as transações usando a fórmula AVG()

In [141]:
try:
    cur13 = conn.cursor()
    cur13.execute("""WITH tabAux AS (SELECT * FROM db_hiring_test.raw_transactions_table t1 INNER JOIN db_hiring_test.raw_transactions_fee t2 ON REPLACE(t1.TotalSent,',','') <= t2.`range-end` AND REPLACE(t1.TotalSent,',','') >= t2.`range-start`) 
                    SELECT ROUND(AVG(REPLACE(TotalSent,',','')*`fee-percentage`/100),2) AS AvgTaxPaid FROM tabAux LIMIT 1""") 
    query_results13 = cur.fetchall()
    field_names = print([i[0] for i in cur13.description])
    for row in query_results13:
      print(row)
except Exception as e:
    print("Database connection failed due to {}".format(e))

['AvgTaxPaid']
(21713.32,)


# Fase 3:

Para criação da estrutura da automação dentro do Google Cloud Platform, para rodar os scripts podemos utilizar as ferramentas:

*   Google IAM
*   Google Cloud Function
*   Google Cloud Scheduler

Primeiramente podemos criar uma **service account** e posteriormente uma **cloud function** contendo o código em Python que possui a função que cria, altera ou consulta o nosso banco de dados, como por exemplo uma das células acima. Durante a criação, podemos colocar como **trigger** uma requisição HTTP, adicionando a nossa service account como um dos usuários com permissão para acessá-la.

Logo depois da criação da cloud function, devemos criar um **Cloud Scheduler**, lá definiremos a frequência e também o endereço HTTP que o scheduler irá realizar a requisição. É necessário incluir o permissionamento juntamente com a service account previamente criada. 

É possível tornar a Cloud Function pública, evitando a primeira etapa do IAM, porém não se trata da melhor prática, pois qualquer pessoa da web poderia realizar a requisição.